# Example Case Study: Simple PID Control

This notebook is a conversion of the example script `examples/case_01_simple_pid_control.py`. It demonstrates how to use the `SimulationManager` with a single configuration dictionary to set up and run a complete closed-loop control simulation.

## 1. Goal of the Simulation

The goal is to control the water level in a simple reservoir. 

- A **Reservoir** starts with a water level of 0.0m.
- A **PID Controller** has a target setpoint of 10.0m.
- The controller will adjust the **inflow** to the reservoir to try to reach the setpoint.
- A constant **outflow** of 0.5 m³/s is always being drawn from the reservoir.

This setup mimics a common real-world scenario, like a water tower that needs to be kept full while users are drawing water from it.

## 2. Imports and Setup

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import pandas as pd

# Add the project root to the path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

# In the original script, this was `water_system_simulator.simulation_manager`.
# We use the new refactored path here.
from water_system_sdk.src.chs_sdk.simulation_manager import SimulationManager

## 3. The Simulation Configuration Dictionary

The entire simulation is defined in a single Python dictionary. This approach is powerful because the configuration can be easily saved to or loaded from a file (like a YAML or JSON).

In [ ]:
pid_control_config = {
    # --- Simulation Parameters ---
    # Defines the total duration and the time step for the simulation.
    "simulation_params": {
        "total_time": 100, # seconds
        "dt": 0.1      # seconds
    },
    
    # --- Components ---
    # This section defines all the building blocks of our simulation.
    "components": {
        "reservoir_A": {
            "type": "ReservoirModel", # This needs to be a key in the ComponentRegistry
            "params": {
                "area": 1.0,
                "initial_level": 0.0
            }
        },
        "pid_controller": {
            "type": "PIDController",
            "params": {
                "Kp": 2.0,
                "Ki": 0.5,
                "Kd": 1.0,
                "set_point": 10.0
            }
        },
        # We use a TimeSeriesDisturbance with a single point to create a constant outflow.
        "constant_outflow": {
            "type": "TimeSeriesDisturbance",
            "params": {
                "times": [0],   
                "values": [0.5] 
            }
        }
    },
    
    # --- Connections ---
    # This defines how the components are wired together. 
    # It copies the value from a 'source' to a 'target' at the start of each time step.
    "connections": [
        {
            "source": "reservoir_A.state.level",
            "target": "pid_controller.input.error_source"
        },
        {
            "source": "constant_outflow.output",
            "target": "reservoir_A.input.release_outflow"
        }
    ],
    
    # --- Execution Order ---
    # This defines the exact sequence of operations at each time step.
    "execution_order": [
        # 1. PID calculates inflow based on last step's reservoir level.
        # The result is directly written to the reservoir's inflow input.
        {
            "component": "pid_controller",
            "method": "step",
            "args": {"dt": "simulation.dt"},
            "result_to": "reservoir_A.input.inflow"
        },
        # 2. Step the outflow component to ensure its output is updated.
        {
            "component": "constant_outflow",
            "method": "step",
            "args": {"dt": "simulation.dt", "t": "simulation.t"}
        },
        # 3. Finally, step the reservoir, which will now use the new inflow value.
        {
            "component": "reservoir_A",
            "method": "step",
            "args": {"dt": "simulation.dt"}
        }
    ],
    
    # --- Logger Configuration ---
    # Tells the manager which variables to record at each time step.
    "logger_config": [
        "reservoir_A.state.level",
        "pid_controller.state.output",
        "reservoir_A.input.inflow"
    ]
}

## 4. Running the Simulation and Plotting Results

In [ ]:
# Instantiate the manager and run the simulation with the config
manager = SimulationManager(pid_control_config)
results_df = manager.run()

# Display the first few rows of the results
print("Simulation Results:")
print(results_df.head())

# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(results_df['time'], results_df['reservoir_A.state.level'], label='Water Level')
plt.axhline(y=10.0, color='r', linestyle='--', label='Setpoint')
plt.xlabel('Time (s)')
plt.ylabel('Water Level (m)')
plt.title('Simple PID Control Simulation')
plt.legend()
plt.grid(True)
plt.show()

The plot shows the PID controller successfully driving the water level towards the 10m setpoint, demonstrating a stable, closed-loop control system configured entirely from a single dictionary.